#### Install Required Packages
- The script installs and sets up the `libtorrent` library and its dependencies automatically.  
- Make sure to run the install cells first before using the torrent downloader

In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

#### Initialize Session & Mount Google Drive / (If it fail, run again.)
- The script mounts your Google Drive to save downloaded files directly there.  
- You will be prompted to authorize access, Allow it.

In [ ]:
import libtorrent as lt
import time
import requests
import os
from IPython.display import display
import ipywidgets as widgets
from google.colab import drive

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

# === Initialize Torrent Session (faster peer discovery)
ses = lt.session()
ses.listen_on(6881, 6891)
ses.start_dht()
ses.start_lsd()
ses.start_upnp()
ses.start_natpmp()
downloads = []

# === Mount Google Drive
drive.mount("/content/drive")

#### Input Torrent or Magnet Link
- You can paste either a direct `.torrent` file, File URL or a magnet link when asked.  
- Downloads will be saved in your Google Drive under:  `/My Drive/Torrent`  
- You can change this path in the script if needed.
- The script shows a progress bar for each active download with speed, progress %, ETA, and status.


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import libtorrent as lt
import requests
import os

save_path_root = "/content/drive/My Drive/Torrent"
os.makedirs(save_path_root, exist_ok=True)

# Create widgets
mode_selector = widgets.ToggleButtons(
    options=["Paste Link", "Browse File"],
    description="Select Method: ",
    disabled=False,
    button_style="",  # 'success', 'info', 'warning', 'danger' or '',
    layout=widgets.Layout(width='600px'),  # widen the toggle buttons
    style={'description_width': 'initial'}
)

link_input = widgets.Text(
    description="Link:",
    placeholder="Paste magnet or .torrent URL here",
    layout=widgets.Layout(width="70%"),
)

file_upload = widgets.FileUpload(
    accept=".torrent",
    multiple=False,
)

start_button = widgets.Button(description="Start Download")

output = widgets.Output()

# Container for link or file input
input_box = widgets.VBox()

def on_mode_change(change):
    with output:
        clear_output()
    if change['new'] == "Paste Link":
        input_box.children = [link_input, start_button]
    else:
        input_box.children = [file_upload, start_button]

mode_selector.observe(on_mode_change, names='value')

def on_start_clicked(b):
    with output:
        clear_output()
        print("Starting download...")
        downloads = []
        ses = lt.session()
        ses.listen_on(6881, 6891)
        ses.start_dht()
        ses.start_lsd()
        ses.start_upnp()
        ses.start_natpmp()
        
        if mode_selector.value == "Paste Link":
            link = link_input.value.strip()
            if not link:
                print("❌ Please enter a magnet or torrent link.")
                return

            if link.endswith(".torrent") or "torrent/download" in link:
                print("🔗 Detected .torrent file link, downloading...")
                r = requests.get(link)
                torrent_file_path = "/content/temp.torrent"
                with open(torrent_file_path, "wb") as f:
                    f.write(r.content)
                ti = lt.torrent_info(torrent_file_path)
                save_path = save_path_root
                params = {"save_path": save_path, "ti": ti}
                downloads.append(ses.add_torrent(params))
            else:
                print("🧲 Detected magnet link...")
                save_path = save_path_root
                params = {"save_path": save_path}
                downloads.append(lt.add_magnet_uri(ses, link, params))

        else:  # Upload .torrent file mode
            if len(file_upload.value) == 0:
                print("❌ Please upload a .torrent file first.")
                return
            for uploaded_filename in file_upload.value:
                content = file_upload.value[uploaded_filename]['content']
                torrent_file_path = f"/content/{uploaded_filename}"
                with open(torrent_file_path, "wb") as f:
                    f.write(content)
                ti = lt.torrent_info(torrent_file_path)
                save_path = save_path_root
                params = {"save_path": save_path, "ti": ti}
                downloads.append(ses.add_torrent(params))
        
        # Progress bar setup (simplified for demo)
        import time
        import ipywidgets as widgets
        state_str = [
            "queued",
            "checking",
            "downloading metadata",
            "downloading",
            "finished",
            "seeding",
            "allocating",
            "checking fastresume",
        ]

        bars = []
        labels = []

        for _ in downloads:
            bar = widgets.FloatProgress(min=0, max=100, layout=widgets.Layout(width='60%'))
            label = widgets.Label(value="")
            bars.append(bar)
            labels.append(label)

        from IPython.display import display
        for bar, label in zip(bars, labels):
            display(widgets.HBox([bar, label]))

        #display(*bars)
        
        while downloads:
            next_shift = 0
            for i, d in enumerate(downloads[:]):
                s = d.status()
                eta = ((s.total_wanted - s.total_done) / s.download_rate) if s.download_rate > 0 else 0
                eta_str = f"{eta/60:.1f} min" if eta > 0 else "∞"
                desc = f"{d.name()} | {s.download_rate/1000:.1f} kB/s | {s.progress*100:.1f}% | ETA: {eta_str} | {state_str[s.state]}"
                bars[i].value = s.progress * 100
                labels[i].value = desc

# Initialize with link input visible
input_box.children = [link_input, start_button]

display(mode_selector)
display(input_box)
display(output)

start_button.on_click(on_start_clicked)

#### Auto Unmount Drive
- After finishing, you can safely unmount Google Drive to avoid sync issues:

In [ ]:
drive.flush_and_unmount()
print("✅ Google Drive unmounted!")